In [3]:
from flask import Flask, request
import json
import requests
from openai import OpenAI

app = Flask(__name__)

PAGE_ACCESS_TOKEN = "" #inserir o token
LATEST_API_VERSION = "v19.0"
API = "https://graph.facebook.com/"+LATEST_API_VERSION+"/me/messages?access_token="+PAGE_ACCESS_TOKEN

chave = "" # inserir a chave
assis_id = "" # inserir o assistant id
openai_client = OpenAI(api_key=chave)

@app.route('/hello/', methods=['GET', 'POST'])
def hello_world():
    return "Hello World at localhost:8080"

def send_message(sender_id, message_text):
    '''
    Sending response back to the user using facebook graph API
    '''
    r = requests.post("https://graph.facebook.com/"+LATEST_API_VERSION+"/me/messages",
                      params={"access_token": PAGE_ACCESS_TOKEN},
                      headers={"Content-Type": "application/json"},
                      data=json.dumps({
                          "recipient": {"id": sender_id},
                          "message": {"text": message_text}
                      }))
    print(r.text)

@app.route('/', methods=['GET'])
def handle_verification():
    if (request.args.get('hub.verify_token', '') == "chatescola2024"):
        print("Verified")
        return request.args.get('hub.challenge', '')
    else:
        print("Wrong token")
        return "Error, wrong validation token"

@app.route('/', methods=['POST'])
def handle_message():
    '''
    Handle messages sent by Facebook Messenger to the application
    '''
    data = request.get_json()
    print(data)
    if data["object"] == "page":
        for entry in data["entry"]:
            for messaging_event in entry["messaging"]:
                if messaging_event.get("message"):
                    sender_id = messaging_event["sender"]["id"]
                    recipient_id = messaging_event["recipient"]["id"]
                    try:
                        message_text = messaging_event["message"]["text"]
                        # Send the received message to OpenAI API
                        thread = openai_client.beta.threads.create(messages=[{"role": "user",
                                                                              "content": message_text}])
                        run = openai_client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assis_id)
                        while run.status != "completed":
                            run = openai_client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
                        else:
                            message_response = openai_client.beta.threads.messages.list(thread_id=thread.id)
                            messages = message_response.data
                            response = messages[0]
                            response_text = response.content[0].text.value
                            send_message(sender_id, response_text)
                    except Exception as e:
                        print("Error:", e)
    return "ok"

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8080)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8080
 * Running on http://192.168.1.72:8080
Press CTRL+C to quit
127.0.0.1 - - [20/Jun/2024 17:53:11] "GET / HTTP/1.1" 200 -


Wrong token


127.0.0.1 - - [20/Jun/2024 17:53:11] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [20/Jun/2024 17:53:15] "GET / HTTP/1.1" 200 -


Wrong token
